In [2]:
import pickle
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from sklearn.neighbors import NearestNeighbors
import cv2

In [3]:
features_list = np.array(pickle.load(open('embeddings.pkl', 'rb')))
filenames = pickle.load(open('filenames.pkl', 'rb'))

In [4]:
model = ResNet50(weights='imagenet', include_top=False,input_shape=(224,224,3))
model.trainable = False
model = tf.keras.Sequential([
	model,
	GlobalMaxPooling2D()
])

In [5]:

img = image.load_img(path="sample/18.3.jpg", target_size=(224, 224))
img_array = image.img_to_array(img)
expanded_img_array = np.expand_dims(img, axis=0)
preprocessd_img = preprocess_input(expanded_img_array)
result = model.predict(preprocessd_img).flatten()
normalized_result = result / np.linalg.norm(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


In [6]:
neighbours = NearestNeighbors(n_neighbors=10, algorithm='brute', metric='euclidean')
neighbours.fit(features_list)
distances,indices = neighbours.kneighbors([normalized_result])

In [7]:
print(indices)

[[  7  37  35   4  36   6 167  22 166  27]]


In [8]:
for i in indices[0][1:10]:
	temp_img = cv2.imread(filenames[i])
	cv2.imshow('image',cv2.resize(temp_img,(512,512)))
	cv2.waitKey(0)